In [1]:
# 크롤링 라이브러리 임포트
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

companyNameList=[] # 회사명 리스트
companySalaryUrlList = [] # 회사 연봉정보의 Url 리스트
avgCurrencyList = [] # 회사 평균 연봉정보 리스트
avgUnivCurrencyList = [] # 회사 대졸 초임 평균 연봉정보 리스트

# 사람인 연봉정보 사이트 정보 입력
html = urlopen("https://www.saramin.co.kr/zf_user/salaries/total-salary/list?page=1&order=rank&industry_cd=&company_cd=&rec_status=&search_company_nm_org=&search_company_nm=&min_salary=&max_salary=&request_modify_company_nm=")
bsObject = BeautifulSoup(html,"html.parser")

In [2]:
# 해당하는 회사 명 크롤링하기
for i in range(20):
    companyName = bsObject.find_all("a",{"class":"link_tit"})[i].text
    companyNameList.append(companyName)

In [3]:
# 랭킹 1등부터 20등까지의 사람인 연봉 회사의 url 정보 추출
for i in range(20):
    link = "http://www.saramin.co.kr" + bsObject.find_all("a",{"class":"logo"})[i].get("href")
    companySalaryUrlList.append(link)

In [4]:
# 추출한 url에서 평균 연봉정보를 avgCurrencyList에 저장
for i in range(20):
    html2 = urlopen(companySalaryUrlList[i])
    bsObject2 = BeautifulSoup(html2,"html.parser")
    avgCurrency = (bsObject2.find_all("em")[0]).text
    avgCurrency = avgCurrency.replace(",","")
    avgCurrencyList.append(avgCurrency)

In [5]:
# 추출한 정보를 최종 finalList에 저장한 후 csv파일로 저장
arr=[]
arr={
    "회사명":companyNameList,
    "연봉정보":avgCurrencyList}

finalList = pd.DataFrame(arr ,columns = ["회사명","연봉정보"])

In [11]:
print(finalList)

         회사명  연봉정보
0       삼성전자  7271
1   에스케이하이닉스  6413
2      SK텔레콤  7504
3        오뚜기  4312
4        코스콤  6881
5    SK이노베이션  7069
6     한국방송공사  6549
7       농협은행  6290
8      유한킴벌리  6925
9    마이다스아이티  5313
10     이랜드건설  5065
11     현대자동차  6669
12   삼성에스디에스  6944
13    대방산업개발  4770
14     금융결제원  6882
15    한국남방개발  5818
16       포스코  6287
17    울산알루미늄  6643
18       세메스  6693
19      세아상역  5301


In [7]:
# CSV 파일로 저장하기
finalList.to_csv("sample.csv", encoding="utf-8")

In [8]:
#데이터 베이스에 저장하기
import cx_Oracle
import os

LOCATION = r"C:\instantclient_19_11"
os.environ["PATH"]=LOCATION + ";"+os.environ["PATH"]

In [9]:
connect = cx_Oracle.connect("HR","1234","localhost")
cursor = connect.cursor()

In [10]:
for i in range(20):
    sql = "INSERT INTO company VALUES('" + companyNameList[i] +"',"+ avgCurrencyList[i] + ")"
    cursor.execute(sql)
cursor.close()
connect.commit()
connect.close()